
## Peer-graded Assignment: Подготовка данных для построения модели

На прошлой неделе мы познакомились с задачей и данными, пришло время подготовить датасет для построения моделей. Пора задуматься о том, с какими особенностями и трудностями нам предстоит работать в процессе построения модели прогнозирования оттока.

В этом задании нужно подготовиться к последующему процессу построения модели и оценки её качества. Это важный шаг в процессе построения и оптимизации модели, а также выбора и оценки финального решения.

До того, как вы непосредственно начнете что-то обучать, важно понять, а как, вообще говоря, это оценивать? Как лучше грамотно организовать процедуру оценки, какие узкие места могут быть? Часто, адекватная оценка качества бывает не менее важной, чем непосредственно разработка признаков, обучение модели и оптимизация её параметров.

Задание будет оцениваться на основании развернутых ответов на поставленные вопросы.

## Инструкции
1. Начнем с простого, но важного шага. Отделите небольшую выборку от существующих данных. Назовем её hold-out dataset. Эта выборка нужна для контроля качества решения: она не должна использоваться вплоть до контроля качества решения. Наличие такой выборки поможет убедиться, что в процессе моделирования не было допущено ошибок, не произошло переобучение.

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings("ignore")

In [23]:
x_data = pd.read_csv('orange_small_churn_data.train.txt')
y_data = pd.read_csv('orange_small_churn_labels.train.txt', header=None)
y_data.columns = ['Churn']
y_data['Churn'] = [0 if x == -1 else x for x in y_data['Churn']]

In [24]:
Na_columns = []

for column in x_data.columns: # Колонки, полностью состоящие из NaN
    if x_data[column].isna().sum() == x_data.shape[0]:
        x_data.drop(column, axis=1,inplace=True)
        Na_columns.append(column)

x_data.loc[:,'Var191':] = x_data.loc[:,'Var191':].fillna('Na_cat') # Замена NaN категор.признаков соотв.категорией

# Колонки с полностью отсутствующими значениями

In [25]:
X_train, X_hold_out, y_train, y_hold_out = train_test_split(x_data, 
                                                    y_data,
                                                    test_size=0.1, shuffle=False) 

In [26]:
for column in X_train.loc[:, 'Var1':'Var190']:
    median = X_train[column].dropna()[X_train[column].dropna() != 0].median()
    X_train[column].fillna(median, inplace=True)

2. Подумайте и предложите несколько способов (не менее 3х) обработки категориальных признаков, для того, чтобы их можно было использовать при построении модели. Обратите внимание на модуль sklearn.preprocessing. Начать поиски можно с sklearn.preprocessing.OneHotEncoder.


In [27]:
le = LabelEncoder()
for column in X_train.loc[:, 'Var191':]:
    X_train[column] = le.fit_transform(X_train[column])

In [28]:
X_train

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,16.0,5.0,138.0,9.0,240545.0,3052.0,7.0,28.0,573489.0,8.0,...,2725,0,3534,0,1,1,14,0,29,0
1,16.0,5.0,138.0,9.0,240545.0,1813.0,7.0,28.0,573489.0,8.0,...,2733,4,335,0,1,0,22,2,2,2
2,16.0,5.0,138.0,9.0,240545.0,1953.0,7.0,28.0,573489.0,8.0,...,312,6,2380,0,1,1,7,3,25,2
3,16.0,5.0,138.0,9.0,240545.0,1533.0,7.0,28.0,573489.0,8.0,...,3663,4,2480,0,1,1,22,2,8,0
4,16.0,5.0,138.0,9.0,240545.0,686.0,7.0,28.0,573489.0,8.0,...,1575,4,1300,0,1,1,12,2,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35995,16.0,5.0,138.0,9.0,240545.0,7777.0,7.0,28.0,573489.0,8.0,...,1825,4,685,0,1,1,16,2,14,0
35996,16.0,5.0,138.0,9.0,240545.0,252.0,7.0,28.0,573489.0,8.0,...,2672,4,960,0,1,2,1,2,8,1
35997,16.0,5.0,138.0,9.0,240545.0,840.0,0.0,28.0,573489.0,8.0,...,2987,0,2471,0,1,1,4,0,8,0
35998,16.0,5.0,138.0,9.0,240545.0,217.0,7.0,28.0,573489.0,8.0,...,1642,4,2523,0,1,1,2,2,8,0



3. Подумайте, с помощью какой метрики качества лучше всего оценивать качество будущей модели, какой будет ключевая метрика качества? Поясните свой выбор.

Несбалансированный датасет - неравные классы - метрика accuracy бесполезна.

В рамках задачи оттока нам важнее найти клиентов, которые собираются от нас уйти, чем найти клиентов, которые от нас уходить не собираются. Поэтому если мы ошибочно отнесем клиента, который уходить не собирается, к оттоку, это не так страшно, как если мы пропустим клиента, который от нас уходить собирается, потому что в этом случае мы, конечно же, не сможем его удержать.

Другими словами нам нестрашно сделать ложное срабатывание (FP), т.е. выбирается метрика **Полнота (recall)** - нужно меньше ложных пропусков (FN).

4. Какие вспомогательные метрики качества имеет смысл считать, чтобы более полно оценить качество модели, понять, где её слабые стороны и что нужно улучшать?

Имеет смысл дополнительно рассматривать показатель **F1-меры**, метрику которая объединяет в себе информацию о точности и полноте нашего алгоритма (помогает принимать решение о том какую реализацию запускать в prod), а также показатель **Roc-Auc**, демонстрирующий качество классификации

5. Подберите оптимальную стратегию проведения кросс-валидации: решите, на сколько фолдов вы будете делить выборку? Выберите тип разбиения данных (k-fold, stratified k-fold, shuffle split, leave one out). Поясните ваш выбор.

Выбирается стратегия **Stratified K-Fold**